In [1]:
import numpy as np
%matplotlib notebook
import pygimli as pg
from pygimli.viewer import pv
from pygimli.physics.gravimetry import MagneticsModelling

In [2]:
dx = 50
x = np.arange(0., 1001, dx)
y = np.arange(0., 1001, dx)
z = np.arange(0., 501, dx)
grid = pg.createGrid(x, y, z)
print(grid)
grid["number"] = np.arange(grid.cellCount())

Mesh: Nodes: 4851 Cells: 4000 Boundaries: 12800


In [3]:
v = np.zeros((len(z)-1, len(y)-1, len(x)-1))
for i in range(7):
    v[1+i, 11-i:16-i, 7:13] = 0.05

grid["synth"] = v.ravel()

In [4]:
pl, _ = pg.show(grid, "synth", style="wireframe", hold=True)
ftr = dict(value=0.03, scalars="synth")
pv.drawMesh(pl, grid, label="synth", style="surface", filter={"threshold": ftr})
pl.camera_position = "yz"
pl.camera.roll = 90
pl.camera.azimuth = 180
pl.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'643174db-d07f-41c7-8ee2-f1c0e3cc20da': {'defs': …

In [5]:
from scipy.io import loadmat
mat = loadmat("data_LiOldenburg.mat")["data"]
py, px, pz, TFA, Bx, By, Bz, Bxx, Bxy, Bxz, Byy, Byz, Bzz = mat.T
points = np.column_stack((px, py, -pz))
F, I, D = 50000e-9, 75, 25
H = F * np.cos(np.deg2rad(I))
X = H * np.cos(np.deg2rad(D))
Y = H * np.sin(np.deg2rad(D))
Z = F * np.sin(np.deg2rad(I))
igrf = [D, I, H, X, Y, Z, F]

In [6]:
fak = F / (4*np.pi)
data = TFA * fak
cmp = ['TFA']
allcmp = False
if allcmp:
    cmp = ['Bx', 'By', 'Bz']
    data = np.concatenate([Bx, By, Bz]) * fak

In [7]:
# The forward operator
fop = MagneticsModelling(grid, points, cmp, igrf)
model = pg.Vector(grid.cellCount(), 1.0)
response = fop.response(model)
noise_level = 1e-9
response += np.random.randn(len(response)) * noise_level

   0%|          |0/441 [00:00 < ?]

In [8]:
# %% depth weighting
bz = np.array([b.center().z() for b in grid.boundaries() if not b.outside()])
# bn = np.array([b.norm().z() for b in grid.boundaries() if not b.outside()])
z0 = 25
wz = 10 / (bz+z0)**1.5  # * (1 - 0.8*bn)

In [9]:
# %% run inversion
inv = pg.Inversion(fop=fop, verbose=True)  # , debug=True)
inv.setRegularization(correlationLengths=[500, 500, 100])
inv.setRegularization(limits=[0, 0.07])
inv.setConstraintWeights(wz)
startModel = pg.Vector(grid.cellCount(), 0.001)
invmodel = inv.run(data, absoluteError=noise_level, lam=10000.,  # zWeight=0.3,
                   startModel=startModel, # startModel=1e-3, maxIter=21,
                   verbose=True)
grid["inv"] = invmodel

10/05/23 - 20:11:12 - pyGIMLi - INFO - Starting inversion.


fop: <pygimli.physics.gravimetry.MagneticsModelling.MagneticsModelling object at 0x0000020B2F3057C0>
Data transformation: <pgcore._pygimli_.RTrans object at 0x0000020B4705A520>
Model transformation (cumulative):
	 0 <pgcore._pygimli_.RTransLogLU object at 0x0000020B4A81A340>
min/max (data): -7.7e-08/5.6e-07
min/max (error): 1.18%/1528%
min/max (start model): 1.0e-03/1.0e-03
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
inv.iter 1 ... chi² = 1613.15 (dPhi = -37.23%) lam: 10000.0
--------------------------------------------------------------------------------
inv.iter 2 ... chi² = 309.23 (dPhi = 80.71%) lam: 10000.0
--------------------------------------------------------------------------------
inv.iter 3 ... chi² = 28.87 (dPhi = 89.98%) lam: 10000.0
--------------------------------------------------------------------------------
inv.iter 4 ... chi² = 10.65 (dPhi = 59.23%)

In [10]:
ftr = dict(value=0.03, scalars="synth")
pl, _ = pg.show(grid, label="synth", style="wireframe", filter={"threshold": ftr}, hold=True)
ftr = dict(value=0.03, scalars="inv")
pv.drawMesh(pl, grid, label="inv", style="surface", filter={"threshold": ftr})
pl.camera_position = "yz"
pl.camera.roll = 90
pl.camera.azimuth = 180
pl.show()

BokehModel(combine_events=True, render_bundle={'docs_json': {'e7282e71-d657-4bb6-a150-01235cd22f32': {'defs': …

## References
* Li, Y. & Oldenburg, D. (1996): 3-D inversion of magnetic data. Geophysics 61(2), 394-408.
* Holstein et al. (2007): Gravimagnetic field tensor gradiometry formulas for uniform polyhedra, SEG Ext. Abstr.